# Pattern recognition CW2

In [1]:
from scipy.io import loadmat
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import time
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import metric_learn
from numpy import linalg as calc_eigen
from sklearn.preprocessing import normalize as normalize_vectors
import pandas as pd
import pylab as pl
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn import preprocessing
from scipy.spatial.distance import cdist
from scipy.spatial.distance import pdist
from scipy.optimize import minimize

### Import file with 6 main components:

In [2]:
""" 
camId : which camera was used to get the shot (1 or 2)
filelist: names of the images (with format x_label_camId_index.png)
labels: class of the image (which person's image is it?)
query_idx: indexes of test set
gallery_idx: indexes of test set used for kNN 
train_idx: indexes of training and validation set
"""   
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')

### Import file with the feature vectors of all images:

In [3]:
import json
with open('PR_data/feature_data.json', 'r') as f:
    features = json.load(f)
features_np = np.array(features) #list of features converted to an array 
features_np.shape

(14096, 2048)

In [4]:
gallery_indices = train_idxs['gallery_idx'].flatten() - 1
gallery_images = features_np[gallery_indices]
gallery_labels = train_idxs['labels'][gallery_indices]

query_indices = train_idxs['query_idx'].flatten() - 1
query_images = features_np[query_indices]
query_labels = train_idxs['labels'][query_indices]

training_indices = train_idxs['train_idx'].flatten() - 1
training_images = features_np[training_indices]
training_labels = train_idxs['labels'][training_indices]

In [5]:
def calculateAverageFace(X):
    """
    takes the training data set as input 
    and returns the average face
    """
    average_face= np.mean(X, axis=0)
    #plt.figure()
    #plt.title("Average Face")
    #plt.imshow(np.resize(average_face, (46, 56)).T, cmap='gray')
    return average_face

def normalize(X, AvgFace):
    """
    takes the training data set and average face as input 
    and returns the normalized training data set

    """
    Q = np.empty((len(X[:,0]),len(X[0, :])))
    for index, face in enumerate(X):
        Q[index] = face - AvgFace
    return Q.T

def calculateCovarianceMatrix(Q):
    return np.matmul(Q,Q.T)/len(Q[0,:])

def calculateLowDimCovarianceMatrix(Q):
    return np.matmul(Q.T,Q)/len(Q[0,:])

def calculateEigenValuesAndVectors(S):
    """
    Takes covariance matrix as input
    and returns eigen values and vectors
    """
    v, w =  calc_eigen.eigh(S)
    #eigen_vectors[:,i] --> eigen_values[i]
    #eigen vector corresponding to eigen value

    #flips left to right... ascending to descending
    v = np.flip(v, axis=0) #turn ascending into descending
    w = np.flip(w, axis=1) #turn ascending into descending
    return v, w

def calculateWeights(Q, U):
    """
    takes input the normalized input and the eigen space
    outputs the weights of the normalized input
    """
    N = len(Q[0,:])
    W = np.empty((N,len(U[0, :])))
    for index, image in enumerate(Q.T):
        W[index] = np.matmul(image, U)
    return W

def calculateWeights2(Q, U):
    """
    alternative method to calculate weights
    """
    N = len(Q[0,:])
    W2 = np.empty((N,len(U[0, :])))
    W2 = np.matmul(Q.T, top_eigen_vectors)
    return W2

def printImage(face, title, saved_file):
    """
    takes input as the face you want to print, the title of the image, and the location of the file
    you would like to save the image to. 
    """
    plt.figure()
    plt.imshow(np.resize(face, (46, 56)).T, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.savefig(saved_file, bbox_inches='tight')   # save the figure to file
    plt.show()
    plt.close() 
    
def plotEigenValueGraph(v, points):
    """
    plots eigen values against inrementing the number of eigen values used
    in descending order
    """
    y_points = [value for value in v[:points]]
    x_points = [i for i in range(points)]
    plt.xlabel('Number of Eigen Values')
    plt.ylabel('Eigen Values')
    plt.title('Plot of Eigen Values in Descending Order')
    plt.plot(x_points, y_points)
    plt.savefig('eigenvalues.png', bbox_inches='tight')   # save the figure to file
    plt.show()
    plt.close() 

### PCA on Training, Gallery, and Query Set

In [6]:
def PCA(X, M):
    AvgFace = calculateAverageFace(X)
    A = normalize(X, AvgFace)
    S = calculateCovarianceMatrix(A)
    eigen_values, eigen_vectors = calculateEigenValuesAndVectors(S)
    x_points = 1000
    #plotEigenValueGraph(eigen_values, x_points)
    variance_captured = np.sum(eigen_values[:M])*100/np.sum(eigen_values)
    print("Variance Captured by top ", x_points, " features is ", variance_captured, ".")
    top_eigen_vectors = eigen_vectors[:, :M]
    reduced_dim_X =  calculateWeights(A, top_eigen_vectors)
    return reduced_dim_X, top_eigen_vectors, AvgFace

In [7]:
M = 100
reduced_training_images, eigen_space, AvgFace = PCA(training_images, M)

Variance Captured by top  1000  features is  92.92050565369442 .


In [8]:
reduced_gallery_images = calculateWeights(normalize(gallery_images, AvgFace), eigen_space)
reduced_query_images = calculateWeights(normalize(query_images, AvgFace), eigen_space)

## kNN Classification for query set on gallery set

In [9]:
def calculate_recall_index(value, recall): #only used in calcAP function below
    for index, recall_val in enumerate(recall):
        if recall_val >= value:
            return index
    
def calcAP(true_label, rank_k_labels):
    correct_list = [true_label == this_label for this_label in rank_k_labels]    
    precision = []
    recall = []
    for i in range(len(rank_k_labels)):
        precision.append(sum(correct_list[:i+1])/len(correct_list[:i+1]))
        recall.append(sum(correct_list[:i+1])/sum(correct_list))
    max_list = []
    for i in range(11):
        recall_index = calculate_recall_index(i/10, recall)
        max_list.append(max(precision[recall_index:]))
    #print(max_list)
    return np.mean(max_list)

In [38]:
def remove_indices(n_indices, query_index):
    query_label_and_camId = (train_idxs['labels'][query_index][0], train_idxs['camId'][query_index][0])
    list_of_gallery_label_and_camId = [(label[0], camId[0]) for label, camId in zip(train_idxs['labels'][n_indices], train_idxs['camId'][n_indices])] 
    final_list = list(filter(lambda a: a != query_label_and_camId, list_of_gallery_label_and_camId))
    return [tup[0] for tup in final_list]

def get_accuracy(k, N):
    list_of_truths = []
    AP_vals = []
    start = time.time() #time tracking - start time of process
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1)
    neigh.fit(gallery_images)   #new_gallery_labels)
    for index, query_index in enumerate(query_indices.tolist()):
        if (index % 100 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, 2048))
        forLoopStart = time.time()
        topN_gallery_indices = (gallery_indices[N_indices[0]]).tolist()
        reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
        AP = calcAP(query_labels[index][0], reduced_topN_labels[:k])
        AP_vals.append(AP)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    #print(acc)
    return acc, np.mean(AP_vals)

In [38]:
N=100
k = [1, 5, 10]
results = []
for k_val in k: 
    print("For rank ", k_val, ": ")
    accuracy, mAP = get_accuracy(k_val, N)
    tup = (k_val, accuracy, mAP)
    results.append(tup)
    print(tup)

For rank  1 : 
Index:  0  & time taken:  0.6330201625823975


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':


Index:  100  & time taken:  11.09302806854248
Index:  200  & time taken:  21.656609058380127
Index:  300  & time taken:  32.17634606361389
Index:  400  & time taken:  42.74071907997131
Index:  500  & time taken:  53.18624424934387
Index:  600  & time taken:  63.734869956970215
Index:  700  & time taken:  74.28422927856445
Index:  800  & time taken:  84.80377697944641
Index:  900  & time taken:  95.39508509635925
Index:  1000  & time taken:  106.03644609451294
Index:  1100  & time taken:  116.58560514450073
Index:  1200  & time taken:  127.08568906784058
Index:  1300  & time taken:  137.68114113807678
(1, 0.47, 0.47)
For rank  5 : 
Index:  0  & time taken:  0.6057331562042236
Index:  100  & time taken:  11.174184083938599
Index:  200  & time taken:  21.870043992996216
Index:  300  & time taken:  32.56023406982422
Index:  400  & time taken:  43.104028940200806
Index:  500  & time taken:  53.7527060508728
Index:  600  & time taken:  64.34500098228455
Index:  700  & time taken:  74.9793412

### Results for kNN: 

k = 1, Accuracy = 0.47, mAP = 0.47

k = 5, Accuracy = 0.6685714285714286, mAP = 0.5272045454545454

k = 10, Accuracy = 0.7492857142857143, mAP = 0.5169191146155432

In [141]:
def get_accuracy_pca(k, N):
    list_of_truths = []
    AP_vals = []
    start = time.time() #time tracking - start time of process
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1)
    neigh.fit(reduced_gallery_images)   #new_gallery_labels)
    for index, query_index in enumerate(query_indices.tolist()):
        if (index % 200 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh.kneighbors(reduced_query_images[index].reshape(1, len(reduced_query_images[0, :])))
        forLoopStart = time.time()
        topN_gallery_indices = (gallery_indices[N_indices[0]]).tolist()
        reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
        AP = calcAP(query_labels[index][0], reduced_topN_labels[:k])
        AP_vals.append(AP)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print(acc)
    return acc, np.mean(AP_vals)

In [142]:
N=100
k = [1, 5, 10]
results2 = []
for k_val in k: 
    print("For rank ", k_val, ": ")
    accuracy, mAP = get_accuracy_pca(k_val, N)
    tup = (k_val, accuracy, mAP)
    results2.append(tup)
    print(tup)

For rank  1 : 
Index:  0  & time taken:  0.02941107749938965


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':


Index:  200  & time taken:  21.079618215560913
Index:  400  & time taken:  42.17799711227417
Index:  600  & time taken:  63.17202019691467
Index:  800  & time taken:  84.27191829681396
Index:  1000  & time taken:  105.21672511100769
Index:  1200  & time taken:  126.23884606361389
0.46714285714285714
(1, 0.46714285714285714, 0.46714285714285714)
For rank  5 : 
Index:  0  & time taken:  0.013506174087524414
Index:  200  & time taken:  20.97689700126648
Index:  400  & time taken:  41.993072271347046
Index:  600  & time taken:  63.114259004592896
Index:  800  & time taken:  84.1421959400177
Index:  1000  & time taken:  105.16470098495483
Index:  1200  & time taken:  126.25219821929932
0.6757142857142857
(5, 0.6757142857142857, 0.5285400432900433)
For rank  10 : 
Index:  0  & time taken:  0.013849973678588867
Index:  200  & time taken:  21.23994207382202
Index:  400  & time taken:  42.65201115608215
Index:  600  & time taken:  63.94959092140198
Index:  800  & time taken:  85.24179100990295


### Results for kNN using PCA: 

k = 1, Accuracy = 0.46714285714285714, mAP = 0.46714285714285714

k = 5, Accuracy = 0.6757142857142857, mAP = 0.5285400432900433

k = 10, Accuracy = 0.7485714285714286, mAP = 0.5152942434549578

## K-means

In [157]:
def img_indices_in_cluster(c, index_and_cluster): 
    gallery_indices_in_cluster = [pair[0] for pair in index_and_cluster if pair[1] == c]
    return gallery_indices_in_cluster

def img_cluster(k, N, c_g_indices, query_index, index):
    list_of_truths = []
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(features_np[c_g_indices])   #new_gallery_labels)
    N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, -1))
    topN_c_g_indices = (gallery_indices[N_indices[0]]).tolist()
    reduced_topN_labels = remove_indices(topN_c_g_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

def knn_clustering(k, N, index_and_cluster):
    class_success = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(query_indices.tolist()[:700]):
        if (index % 200 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        cluster = kmeans.predict(query_images[index].reshape(1,-1))
        cluster_gallery_indices = img_indices_in_cluster(cluster, index_and_cluster)
        class_success.append(img_cluster(k, N, cluster_gallery_indices, query_index, index))
    
    #over-all accuracy
    acc = sum(class_success)/len(class_success)
    print(acc)
    return acc

In [47]:
total_number_of_labels = len(np.unique(gallery_labels.flatten()))

acc_kmeans = []
clusters = [1, 3, 10] 
clusters2 = [100, total_number_of_labels]
for n_clusters in clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(gallery_images)
    index_and_cluster = [(index, label) for index, label in zip(gallery_indices, kmeans.labels_)]
    k = [1, 5, 10]
    N = 100
    for k_val in k:
        acc_kmeans.append(knn_clustering(k_val, N, index_and_cluster)) 

Index:  0  & time taken:  4.506111145019531e-05
Index:  200  & time taken:  150.7624750137329
Index:  400  & time taken:  301.3440308570862
Index:  600  & time taken:  461.696977853775
0.49714285714285716
Index:  0  & time taken:  4.696846008300781e-05
Index:  200  & time taken:  156.14981603622437
Index:  400  & time taken:  307.5661029815674
Index:  600  & time taken:  461.1602611541748
0.69
Index:  0  & time taken:  5.0067901611328125e-05
Index:  200  & time taken:  153.74979496002197
Index:  400  & time taken:  308.7922320365906
Index:  600  & time taken:  465.1946897506714
0.7742857142857142
Index:  0  & time taken:  4.792213439941406e-05
Index:  200  & time taken:  51.17026996612549
Index:  400  & time taken:  104.5521879196167
Index:  600  & time taken:  155.86516094207764
0.0
Index:  0  & time taken:  7.796287536621094e-05
Index:  200  & time taken:  51.24071407318115
Index:  400  & time taken:  104.89248394966125
Index:  600  & time taken:  156.0232548713684
0.0014285714285714

In [158]:
total_number_of_labels = len(np.unique(gallery_labels.flatten()))
acc_kmeans2 = []
clusters = [100, total_number_of_labels]
for n_clusters in clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(gallery_images)
    index_and_cluster = [(index, label) for index, label in zip(gallery_indices, kmeans.labels_)]
    k = [1, 5, 10]
    N = 20
    for k_val in k:
        acc_kmeans2.append(knn_clustering(k_val, N, index_and_cluster)) 

Index:  0  & time taken:  5.2928924560546875e-05
Index:  200  & time taken:  31.98020100593567
Index:  400  & time taken:  60.46903991699219
Index:  600  & time taken:  91.8608329296112
0.0
Index:  0  & time taken:  0.00010395050048828125
Index:  200  & time taken:  31.05888295173645
Index:  400  & time taken:  58.93773698806763
Index:  600  & time taken:  90.08086109161377
0.002857142857142857
Index:  0  & time taken:  0.00021696090698242188
Index:  200  & time taken:  31.06084704399109
Index:  400  & time taken:  59.05705428123474
Index:  600  & time taken:  89.93986010551453
0.005714285714285714
Index:  0  & time taken:  6.794929504394531e-05


ValueError: Expected n_neighbors <= n_samples,  but n_samples = 3, n_neighbors = 20

### Atttempt 1: Mahalanobis Distance Using metric_learn

In [26]:
#S = np.cov(np.transpose(training_images))
#A = np.linalg.inv(S)

In [150]:
def knn_maha(k, N):   
    start = time.time() #time tracking - start time of process
    lmnn = metric_learn.LMNN(k=k)
    # fit the data!
    lmnn.fit(reduced_training_images, training_labels) #reduced_dim_training_images
    # transform our input space
    A = lmnn.metric()
    S = np.linalg.inv(A)

    list_of_truths = []
    i = 0
    AP_vals = []
    for q_index, q_image in zip(query_indices, reduced_query_images): 
        if (i % 100 == 0):
            print("Index: ", i, " & time taken: ", time.time() - start)
        query_distances = []
        for g_index, g_image in zip(gallery_indices, reduced_gallery_images): 
            image_diff = q_image - g_image
            query_distances.append(tuple([g_index, np.linalg.multi_dot([np.transpose(image_diff), S, image_diff])]))
        query_distances_sorted = sorted(query_distances,key=lambda x:(x[1]))
        N_gallery_indices = [tup[0] for tup in query_distances_sorted][:N]
        reduced_N_gallery_labels = remove_indices(N_gallery_indices, q_index)
        if query_labels[i][0] in reduced_N_gallery_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
        AP = calcAP(query_labels[i][0], reduced_N_gallery_labels[:k])
        AP_vals.append(AP)
        i +=1
    acc = sum(list_of_truths)/len(list_of_truths) 
    return acc, np.mean(AP_vals)

In [151]:
k = [1, 5]
N = 50
results3 = []
for k_val in k: 
    print("For rank ", k_val, ": ")
    accuracy, mAP = knn_maha(k_val, N)
    tup = (k_val, accuracy, mAP)
    results3.append(tup)
    print(tup)

For rank  1 : 
Index:  0  & time taken:  5.427803039550781


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in long_scalars
  if sys.path[0] == '':


Index:  100  & time taken:  13.546617984771729
Index:  200  & time taken:  21.883944988250732
Index:  300  & time taken:  29.981092929840088
Index:  400  & time taken:  38.06189799308777
Index:  500  & time taken:  46.22734594345093
Index:  600  & time taken:  54.30570983886719
Index:  700  & time taken:  62.39797401428223
Index:  800  & time taken:  70.60209202766418
Index:  900  & time taken:  78.81504797935486
Index:  1000  & time taken:  86.92118906974792
Index:  1100  & time taken:  96.24939298629761
Index:  1200  & time taken:  104.35860800743103
Index:  1300  & time taken:  112.7339038848877
(1, 0.13, 0.13)
For rank  5 : 
Index:  0  & time taken:  5.489860773086548
Index:  100  & time taken:  13.865935802459717
Index:  200  & time taken:  22.857558965682983
Index:  300  & time taken:  32.80487608909607
Index:  400  & time taken:  40.977351903915405
Index:  500  & time taken:  49.45302486419678
Index:  600  & time taken:  57.52837085723877
Index:  700  & time taken:  66.459969997

### Attempt 2: Mahalanobis Distance Using Cdist

In [223]:
A = np.cov(np.transpose(reduced_training_images))
S = np.linalg.inv(A)

In [233]:
S[-1] #all features after PCA are independant 
#covariance matrix A/S gives us 0 on everything except of the diagonal 
#mahalanobis distance is only useful when features are dependant (why?). whereas, in our case, they are independant. 

array([-6.59143173e-17, -5.26842904e-19,  5.30399860e-17,  3.93850271e-19,
       -1.99653840e-18,  2.90425986e-18,  3.60573004e-17, -3.58023330e-17,
       -2.68850768e-17,  4.33028843e-17,  4.90452495e-17, -8.25971584e-18,
        2.93800106e-17, -6.23003251e-17,  1.74532655e-17, -9.82029995e-17,
        1.03953919e-16,  1.61497025e-17,  8.28539364e-17,  2.46735852e-17,
        1.09153047e-16,  3.31786616e-17, -6.10328075e-17, -1.11164710e-17,
        2.10812941e-17, -6.10108012e-17, -1.67729673e-17,  6.04618801e-17,
        4.67698463e-18,  1.55277336e-16, -1.45465364e-16,  5.10472924e-18,
        3.10928011e-16, -4.08300533e-17,  2.98490139e-16, -1.71100616e-16,
        4.12243474e-16, -8.73094183e-17,  2.68815666e-16, -2.87291729e-17,
       -1.72161936e-17, -2.15361594e-16, -1.39882657e-16,  1.22915095e-16,
       -1.85143611e-16, -1.80502757e-16, -8.02014801e-17, -3.69328462e-16,
       -3.09615432e-17,  3.70857649e-16,  2.68027205e-16, -3.61503404e-16,
       -5.29037920e-16, -

In [224]:
distance_matrix = cdist(reduced_query_images, reduced_gallery_images, 'mahalanobis', VI=S)
distance_matrix.shape #rows represent query images, columns represent gallery images

(1400, 5328)

In [225]:
def add_gallery_indices_to_distance_matrix(distance_matrix):
    new_array = np.zeros((5328,1400),dtype=object)  
    distance_matrix2 = []
    for g_index, g_row in enumerate(distance_matrix.T): 
        for q_index, val in enumerate(g_row):
            tup1 = tuple([gallery_indices[g_index], val])
            new_array[g_index][q_index] = tup1
    return new_array

def calculate_accuracy_from_distance_matrix(new_array, k, N):
    list_of_truths = []
    for q_index, column in enumerate(new_array.T):
        query_distances_sorted = sorted(column,key=lambda x:(x[1]))
        N_gallery_indices = [tup[0] for tup in query_distances_sorted][:N]
        reduced_N_gallery_labels = remove_indices(N_gallery_indices, query_indices[q_index])
        if query_labels[q_index][0] in reduced_N_gallery_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
    acc = sum(list_of_truths)/len(list_of_truths) 
    return acc

def calc_acc_maha(k, N, distance_matrix): 
    arr = add_gallery_indices_to_distance_matrix(distance_matrix)
    acc = calculate_accuracy_from_distance_matrix(arr, k, N)
    return acc

In [226]:
k = 1
N = 50
print(calc_acc_maha(k, N, distance_matrix))

0.41785714285714287


In [227]:
k = 5
N = 50
print(calc_acc_maha(k, N, distance_matrix))

0.6221428571428571


In [228]:
k = 10
N = 50
print(calc_acc_maha(k, N, distance_matrix))

0.7028571428571428


### Attempt 3: Getting distances using kNN and Mahalanobis Distance

In [31]:
N = 100
neigh_md = NearestNeighbors(n_neighbors=N,metric='mahalanobis', 
            metric_params= {'V': np.cov(np.transpose(training_images))})
neigh_md.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='mahalanobis',
         metric_params={'V': array([[ 0.12217,  0.00295, ..., -0.00943, -0.01215],
       [ 0.00295,  0.23106, ...,  0.00179, -0.01196],
       ...,
       [-0.00943,  0.00179, ...,  0.32026,  0.09041],
       [-0.01215, -0.01196, ...,  0.09041,  0.15056]])},
         n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [117]:
def get_accuracy_mahalanobi(k, N):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh_md.kneighbors(query_images[index].reshape(1, 2048), N)
        topN_gallery_images = gallery_images[N_indices[0]]
        forLoopStart = time.time()
        topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
        reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    return acc

In [118]:
N = 100
k = [1, 10, 100]
acc_k2 = []
for k_val in k: 
    acc = get_accuracy_mahalanobi(k_val, N)
    acc_k2.append(acc)

Index:  0  & time taken:  8.082389831542969e-05
Index:  50  & time taken:  1194.6821658611298
Index:  100  & time taken:  2801.5326719284058
Index:  150  & time taken:  4013.9011237621307
Index:  200  & time taken:  5203.121114730835
Index:  250  & time taken:  6391.557485818863
Avg Accuracy is:  0.13666666666666666
Index:  0  & time taken:  7.224082946777344e-05
Index:  50  & time taken:  1187.0171492099762
Index:  100  & time taken:  2373.66286110878
Index:  150  & time taken:  3560.3639900684357
Index:  200  & time taken:  4746.637277126312
Index:  250  & time taken:  5932.8617470264435
Avg Accuracy is:  0.2833333333333333
Index:  0  & time taken:  9.083747863769531e-05
Index:  50  & time taken:  1186.3573276996613
Index:  100  & time taken:  2372.715752840042
Index:  150  & time taken:  3559.498749732971
Index:  200  & time taken:  4745.957722663879
Index:  250  & time taken:  5932.393654823303
Avg Accuracy is:  0.4533333333333333


In [119]:
acc_k2

[0.13666666666666666, 0.2833333333333333, 0.4533333333333333]

### Cosine Distance

In [120]:
N = 100
neigh_c = NearestNeighbors(n_neighbors=N,metric='cosine')
neigh_c.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [126]:
def get_acc_gen(k, N, index, query_index):
    N_distances, N_indices = neigh_c.kneighbors(query_images[index].reshape(1, 2048), N)
    topN_gallery_images = gallery_images[N_indices[0]]
    forLoopStart = time.time()
    topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
    reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

k = [1, 10, 100]
acc_k3 = []
for k_val in k:
    list_of_truths = [] 
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        truth_false = get_acc_gen(k_val, N, index, query_index)
        list_of_truths.append(truth_false)
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    acc_k3.append(acc)

Index:  0  & time taken:  7.700920104980469e-05
Index:  50  & time taken:  4.227119207382202
Index:  100  & time taken:  8.423174142837524
Index:  150  & time taken:  12.630351066589355
Index:  200  & time taken:  16.83267116546631
Index:  250  & time taken:  21.04805612564087
Avg Accuracy is:  0.49
Index:  0  & time taken:  5.1975250244140625e-05
Index:  50  & time taken:  4.1925787925720215
Index:  100  & time taken:  8.413705825805664
Index:  150  & time taken:  12.6040678024292
Index:  200  & time taken:  16.792085886001587
Index:  250  & time taken:  20.98653793334961
Avg Accuracy is:  0.74
Index:  0  & time taken:  5.3882598876953125e-05
Index:  50  & time taken:  4.198216199874878
Index:  100  & time taken:  8.467583894729614
Index:  150  & time taken:  12.67863917350769
Index:  200  & time taken:  16.884459972381592
Index:  250  & time taken:  21.093831062316895
Avg Accuracy is:  0.91


### Optimization 

In [10]:
S = np.linalg.inv(np.cov(np.transpose(reduced_training_images)))
distance_matrix = cdist(reduced_query_images, reduced_gallery_images, 'mahalanobis', VI=S)
distance_matrix.shape #rows represent query images, columns represent gallery images

(1400, 5328)

In [10]:
#What is the aggregate distance constraint? 
def loss_function():
    y = np.zeros((1400, 5328),dtype=object) 
    for q_index, q_image in enumerate(reduced_query_images):
        for g_index, g_image in enumerate(reduced_gallery_images):
            if gallery_labels[g_index] == query_labels[q_index]: 
                y[q_index][g_index] = 1
            else:
                y[q_index][g_index] = 0
    distances = []
    for i in range(len(query_labels)):
        agg_dist = np.dot(y[i, :],distance_matrix.T[:, i])
        distances.append(agg_dist)
    return sum(distances)

#print(loss_function())

In [11]:
def calculate_y(X, Y, X_labels, Y_labels):
    y = np.zeros((len(X[:, 0]), len(Y[:, 0])),dtype=object) 
    for t1_index, t1_image in enumerate(X):
        for t2_index, t2_image in enumerate(Y):
            if X_labels[t1_index] == Y_labels[t2_index]: 
                y[t1_index][t2_index] = 1
            else:
                y[t1_index][t2_index] = 0
    return y

def loss_function_training(A):
    distance_matrix_training = cdist(reduced_training_images, reduced_training_images, 'mahalanobis', VI=A)   
    agg_dist = sum([np.dot(y_train[i, :],distance_matrix_training.T[:, i]) for i in range(len(training_labels))])
    return agg_dist

In [12]:
print(loss_function_training(S))

1
2
345445.9685922416


In [12]:
#training
y_train = calculate_y(reduced_training_images, reduced_training_images, training_labels, training_labels)
#y_test = calculate_y(reduced_query_images, reduced_gallery_images, query_labels, gallery_labels)

In [13]:
A0 = np.linalg.inv(np.cov(np.transpose(reduced_training_images)))
optimization_learning = minimize(loss_function_training, A0, method='CG', options={'maxiter': 1, 'disp' : b True})

In [ ]:
print(optimization_learning.A)